In [3]:
!pip install beautifulsoup4 requests pandas

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install lxml


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
 import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_highest-grossing_films'

# Sending a GET request
response = requests.get(url)


if response.status_code == 200:
   # HTML content parsing
    soup = BeautifulSoup(response.text, 'html.parser')

   # Search for the first table with the 'wikitable' class
    table = soup.find('table', {'class': 'wikitable'})

   # Extracting data from a table
    rows = table.find_all('tr')

    movie_data = []

    for row in rows[1:]:
        columns = row.find_all(['td', 'th'])

        # Check that there are enough cells in the row
        if len(columns) >= 5:
            try:
                rank = columns[0].get_text(strip=True)  # Movie rank
                peak = columns[1].get_text(strip=True)  # Peak
                title_column = columns[2]  # Movie title column
                title = title_column.get_text(strip=True)  # Movie name

                # Extract the link to the movie page
                link = title_column.find('a')
                if link:
                    movie_url = 'https://en.wikipedia.org' + link.get('href')  
                    
                else:
                    movie_url = None
                    

                
                box_office = columns[3].get_text(strip=True)  # Box office fees
                release_year = columns[4].get_text(strip=True)  # Year of release

                
                try:
                    release_year = int(release_year)
                except ValueError:
                    release_year = None  

                director = None  # Director (not found in the table)
                country = None  # Country (not found in the table)


                if movie_url:
                    # Sending a GET request to the movie page
                    movie_response = requests.get(movie_url)

                    if movie_response.status_code == 200:
                        movie_soup = BeautifulSoup(movie_response.text, 'html.parser')

                       # Extracting information about the country and the director
                        
                        infobox = movie_soup.find('table', {'class': 'infobox vevent'})

                        if infobox:
                           # trying out the director
                            director_row = infobox.find('th', string='Directed by')
                            if director_row:
                                director_td = director_row.find_next_sibling('td')
                                if director_td:
                                    director = director_td.get_text(strip=True)

                            # trying out the country
                            country_row = infobox.find('th', string='Countries')
                            if country_row:
                                country_td = country_row.find_next_sibling('td')
                                if country_td:
                                    country = country_td.get_text(strip=True)
                                    


                         # trying out the country
                            country_row = infobox.find('th', string='Country')
                            if country_row:
                                country_td = country_row.find_next_sibling('td')
                                if country_td:
                                    country = country_td.get_text(strip=True)
                                    

                        
                        # Saving the data to the dictionary
                        movie = {
                            'title': title,
                            'release_year': release_year,
                            'director': director,
                            'box_office': box_office,
                            'country': country,
                            'movie_url': movie_url
                        }
                        movie_data.append(movie)
                    else:
                        print(f"Ошибка при получении страницы фильма: {movie_response.status_code}")
                else:
                    print(f"Ссылка на страницу фильма не найдена для: {title}")
            except IndexError:
                print(f"IndexError при обработке строки: {row}")
                continue
            except Exception as e:
                print(f"Ошибка при обработке строки: {row}. Ошибка: {e}")
                continue  # Пропускаем проблемную строку

    # Converting data to a DataFrame
    df = pd.DataFrame(movie_data)

    # Output of the first lines of the table
    print(df.head())

    # Saving data to a CSV file
    df.to_csv('highest_grossing_films.csv', index=False, encoding='utf-8-sig')
else:
    print(f"Ошибка при получении страницы: {response.status_code}")


                          title  release_year                director  \
0                        Avatar          2009           James Cameron   
1             Avengers: Endgame          2019  Anthony RussoJoe Russo   
2      Avatar: The Way of Water          2022           James Cameron   
3                       Titanic          1997           James Cameron   
4  Star Wars: The Force Awakens          2015            J. J. Abrams   

        box_office                            country  \
0   $2,923,706,026  United Kingdom[2]United States[2]   
1   $2,797,501,328                      United States   
2   $2,320,250,281                      United States   
3  T$2,257,844,554                      United States   
4   $2,068,223,624                      United States   

                                           movie_url  
0   https://en.wikipedia.org/wiki/Avatar_(2009_film)  
1    https://en.wikipedia.org/wiki/Avengers:_Endgame  
2  https://en.wikipedia.org/wiki/Avatar:_The_Way_... 

In [9]:
import pandas as pd
import re

# movie_data is assumed to contain data extracted from Wikipedia
df = pd.DataFrame(movie_data)

# Output of the first rows of the table for verification
print(df.head())

# Data cleanup
#1. Removing duplicates (if necessary)
df = df.drop_duplicates()

#2. Deleting rows with missing values (if necessary)
df = df.dropna()

# Clearing the box_office column of currency symbols and extra symbols
def clean_box_office(x):
    result = ''
    for char in x:
        if char.isdigit() or char == ',':
            result += char
    return result

df['box_office'] = df['box_office'].apply(clean_box_office)



# Removing links from text columns
df['country'] = df['country'].apply(lambda x: re.sub(r'\[.*?\]', '', x))


df['director'] = df['director'].str.replace(r'\[.*?\]', '')
# Deleting a column from a URL
del df['movie_url']


#5. Converting a release_year column to an integer type
df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce').astype(int)
print(df.head(5))
# Saving data to a CSV file (optional)
df.to_csv('highest_grossing_films.csv', index=False, encoding='utf-8-sig')


                          title  release_year                director  \
0                        Avatar          2009           James Cameron   
1             Avengers: Endgame          2019  Anthony RussoJoe Russo   
2      Avatar: The Way of Water          2022           James Cameron   
3                       Titanic          1997           James Cameron   
4  Star Wars: The Force Awakens          2015            J. J. Abrams   

        box_office                            country  \
0   $2,923,706,026  United Kingdom[2]United States[2]   
1   $2,797,501,328                      United States   
2   $2,320,250,281                      United States   
3  T$2,257,844,554                      United States   
4   $2,068,223,624                      United States   

                                           movie_url  
0   https://en.wikipedia.org/wiki/Avatar_(2009_film)  
1    https://en.wikipedia.org/wiki/Avengers:_Endgame  
2  https://en.wikipedia.org/wiki/Avatar:_The_Way_... 

In [10]:
import sqlite3
import pandas as pd

# Connecting to the database
conn = sqlite3.connect('films.db')

# Creating a table
conn.execute('''
    CREATE TABLE IF NOT EXISTS films (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        release_year INTEGER,
        director TEXT,
        box_office REAL,
        country TEXT
    )
''')

# Inserting data
df.to_sql('films', conn, if_exists='replace', index=False)

# Fetching all the data from the table
cursor = conn.cursor()
cursor.execute("SELECT * FROM films")

# Getting all the results
rows = cursor.fetchall()

# Table output
for row in rows:
    print(row)

# Closing the connection
conn.close()


('Avatar', 2009, 'James Cameron', '2,923,706,026', 'United KingdomUnited States')
('Avengers: Endgame', 2019, 'Anthony RussoJoe Russo', '2,797,501,328', 'United States')
('Avatar: The Way of Water', 2022, 'James Cameron', '2,320,250,281', 'United States')
('Titanic', 1997, 'James Cameron', '2,257,844,554', 'United States')
('Star Wars: The Force Awakens', 2015, 'J. J. Abrams', '2,068,223,624', 'United States')
('Avengers: Infinity War', 2018, 'Anthony RussoJoe Russo', '2,048,359,754', 'United States')
('Ne Zha 2†', 2025, 'Jiaozi', '1,963,942,560', 'China')
('Spider-Man: No Way Home', 2021, 'Jon Watts', '1,922,598,800', 'United States')
('Inside Out 2', 2024, 'Kelsey Mann', '1,698,863,816', 'United States')
('Jurassic World', 2015, 'Colin Trevorrow', '1,671,537,444', 'United States')
('The Lion King', 2019, 'Jon Favreau', '1,656,943,394', 'United States')
('The Avengers', 2012, 'Joss Whedon', '1,518,815,515', 'United States')
('Furious 7', 2015, 'James Wan', '1,515,341,399', 'United Sta

In [11]:
import pandas as pd
import json

# Let's say you have a DataFrame df
df.to_json('data.json', orient='records')

with open('data.json', 'r') as file:
    data = json.load(file)
    print(json.dumps(data, indent=4))



[
    {
        "title": "Avatar",
        "release_year": 2009,
        "director": "James Cameron",
        "box_office": "2,923,706,026",
        "country": "United KingdomUnited States"
    },
    {
        "title": "Avengers: Endgame",
        "release_year": 2019,
        "director": "Anthony RussoJoe Russo",
        "box_office": "2,797,501,328",
        "country": "United States"
    },
    {
        "title": "Avatar: The Way of Water",
        "release_year": 2022,
        "director": "James Cameron",
        "box_office": "2,320,250,281",
        "country": "United States"
    },
    {
        "title": "Titanic",
        "release_year": 1997,
        "director": "James Cameron",
        "box_office": "2,257,844,554",
        "country": "United States"
    },
    {
        "title": "Star Wars: The Force Awakens",
        "release_year": 2015,
        "director": "J. J. Abrams",
        "box_office": "2,068,223,624",
        "country": "United States"
    },
    {
        "titl